<a href="https://colab.research.google.com/github/PaulToronto/Stanford-Andrew-Ng-Machine-Learning-Specialization/blob/main/Gradient_Descent_with_Sympy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient Descent with `sympy`

## Imports

In [1]:
import sympy as sym
import pandas as pd
import numpy as np
from math import ceil

## Symbols

In [2]:
# training set data
x11, x12, x13, x14 = sym.symbols('x_{11} x_{12} x_{13} x_{14}')
x21, x22, x23, x24 = sym.symbols('x_{21} x_{22} x_{23} x_{24}')
x31, x32, x33, x34 = sym.symbols('x_{31} x_{32} x_{33} x_{34}')

# target
y1, y2, y3 = sym.symbols('y_1 y_2 y_3')

# weights and bias
w1, w2, w3, w4, b = sym.symbols('w_1 w_2 w_3 w_4 b')

# weight for simple linear regression
m1 = sym.symbols('m_1')

## Toy Datasets

### Dataset with symbols

In [3]:
data = pd.DataFrame({'feature1': [x11, x21, x31],
                     'feature2': [x12, x22, x32],
                     'feature3': [x13, x23, x33],
                     'feature4': [x14, x24, x34],
                     'target': [y1, y2, y3]})

data

,feature1,feature2,feature3,feature4,target
0,x_{11},x_{12},x_{13},x_{14},y_1
1,x_{21},x_{22},x_{23},x_{24},y_2
2,x_{31},x_{32},x_{33},x_{34},y_3


### Dataset with numbers

In [4]:
data_num = pd.DataFrame({'feature1': [2104, 1416, 852],
                         'feature2': [5, 3, 2],
                         'feature3': [1, 2, 1],
                         'feature4': [45, 40, 35],
                         'target': [460, 232, 178]})

data_num

,feature1,feature2,feature3,feature4,target
0,2104,5,1,45,460
1,1416,3,2,40,232
2,852,2,1,35,178


In [5]:
# optimal w and b for testing
wn_best = sym.Matrix(np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618]))
bn_best = 785.1811367994083
wn_best

Matrix([
[  0.39133535],
[ 18.75376741],
[-53.36032453],
[-26.42131618]])

### Dataset for simple linear regression

In [6]:
data_simple = pd.DataFrame({'feature': [1, 2],
                            'target': [300, 500]})
data_simple

,feature,target
0,1,300
1,2,500


In [7]:
# optimal m and b for testing
ms_best = 200
bs_best = 100

## Using `sympy` Matrices to train the model

`X_train` is represented by the matrix $\mathbf{X}$

$$
\mathbf{X} = \begin{bmatrix}
x_{11} & x_{12} & x_{13} & x_{14}\\
x_{21} & x_{22} & x_{23} & x_{24}\\
x_{31} & x_{32} & x_{33} & x_{34}\\
\end{bmatrix}
$$

`y_train` is represented by the column matrix $\mathbf{y}$.

$$
\mathbf{y} = \begin{bmatrix}y_1\\y_2\\y_3\end{bmatrix}
$$

The weights of the model are represented by the matrix $\mathbf{w}$ the bias is represented by the scalar $b$.

$$
\mathbf{w} = \begin{bmatrix}w_1\\w_2\\w_3\\w_4\end{bmatrix}
$$

The goal of linear regression is to find the values for $\mathbf{w}$ and $b$ that minimize the cost. The cost is defined later with the cost funtion, but in a nutshell, the closer the left side of this "equation" is to the right side, the lower the cost. The left side are the model's predictions, $\widehat{y}$, the right side are the actual values for the target, $y$.

$$
\begin{align}
\mathbf{X}\mathbf{w} + b &= \mathbf{y} \\
\begin{bmatrix}
x_{11} & x_{12} & x_{13} & x_{14}\\
x_{21} & x_{22} & x_{23} & x_{24}\\
x_{31} & x_{32} & x_{33} & x_{34}\\
\end{bmatrix}
\begin{bmatrix}w_1\\w_2\\w_3\\w_4\end{bmatrix} +
b\begin{bmatrix}1\\1\\1\end{bmatrix} &= \begin{bmatrix}y_1\\y_2\\y_3\end{bmatrix} \\
\begin{bmatrix}
w_1x_{11} + w_2x_{12} + w_3x_{13} + w_4x_{14} + b\\
w_1x_{21} + w_2x_{22} + w_3x_{23} + w_4x_{24} + b\\
w_1x_{31} + w_2x_{32} + w_3x_{33} + w_4x_{34} + b
\end{bmatrix} &= \begin{bmatrix}y_1\\y_2\\y_3\end{bmatrix}
\end{align}
$$



In [8]:
X = sym.Matrix(data.drop('target', axis=1))
y = sym.Matrix(data['target'])
w = sym.Matrix([w1, w2, w3, w4])

X @ w + b * sym.Matrix([1, 1, 1])

Matrix([
[b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14}],
[b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24}],
[b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34}]])

In [9]:
y

Matrix([
[y_1],
[y_2],
[y_3]])

In [10]:
# dataset with numbers
Xn = sym.Matrix(data_num.drop('target', axis=1))
yn = sym.Matrix(data_num['target'])

In [11]:
# dataset for simple linear regression
Xs = sym.Matrix(data_simple.drop('target', axis=1))
ys = sym.Matrix(data_simple['target'])

## The Model Prediction

$$
f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w}\cdot\mathbf{x}^{(i)} + b
$$

where

- $\mathbf{x}$ is a vector representing the $i^{th}$ row of $\mathbf{X}$
- $\mathbf{w}$ is vector containing the weights of the model
- $b$ is a scalar representing the bias

In [12]:
# `X * w` is used instead of `X @ w`
#   so that the function also works
#   for simple linear regression
def f_wb(X, w, b):
    m = X.shape[0]
    pred = X * w + b * sym.ones(m, 1)
    return pred

In [13]:
# works with a single row
f_wb(X[0,:], w, b)

Matrix([[b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14}]])

In [14]:
f_wb(Xn[0,:], w, b)

Matrix([[b + 2104*w_1 + 5*w_2 + w_3 + 45*w_4]])

In [15]:
# works with multiple rows
f_wb(X, w, b)

Matrix([
[b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14}],
[b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24}],
[b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34}]])

In [16]:
f_wb(Xn, w, b)

Matrix([
[  b + 2104*w_1 + 5*w_2 + w_3 + 45*w_4],
[b + 1416*w_1 + 3*w_2 + 2*w_3 + 40*w_4],
[   b + 852*w_1 + 2*w_2 + w_3 + 35*w_4]])

In [17]:
# works with simple linear regression
f_wb(X[:,0], m1, b)

Matrix([
[b + m_1*x_{11}],
[b + m_1*x_{21}],
[b + m_1*x_{31}]])

In [18]:
f_wb(Xs, m1, b)

Matrix([
[  b + m_1],
[b + 2*m_1]])

## The Cost Function

$$
J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2
$$

In [19]:
def compute_cost_loop(X, y, w, b):
    m = X.shape[0]
    cost = sym.Matrix([0.0])

    for i in range(m):
        f_wb_i = f_wb(X[i,:], w, b)
        cost = cost + (f_wb_i - sym.Matrix([y[i]])).applyfunc(lambda x: x**2)
    cost = cost / (2 * m)
    return cost[0]

In [20]:
# works for multiple linear regression
compute_cost_loop(X, y, w, b)

(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)**2/6 + (b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)**2/6 + (b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)**2/6

In [21]:
compute_cost_loop(Xn, yn, w, b)

(b + 852*w_1 + 2*w_2 + w_3 + 35*w_4 - 178)**2/6 + (b + 1416*w_1 + 3*w_2 + 2*w_3 + 40*w_4 - 232)**2/6 + (b + 2104*w_1 + 5*w_2 + w_3 + 45*w_4 - 460)**2/6

In [22]:
# works for simple linear regression
compute_cost_loop(X[:,0], y, m1, b)

(b + m_1*x_{11} - y_1)**2/6 + (b + m_1*x_{21} - y_2)**2/6 + (b + m_1*x_{31} - y_3)**2/6

In [23]:
compute_cost_loop(Xs, ys, m1, b)

(b + m_1 - 300)**2/4 + (b + 2*m_1 - 500)**2/4

This can also be implemented without the loop.

In [24]:
def compute_cost(X, y, w, b):
    m = X.shape[0]
    pred = f_wb(X, w, b)
    cost = sum((pred - y).applyfunc(lambda x: x**2)) / (2 * m)
    return cost

In [25]:
# works for mulitiple linear regression
compute_cost(X, y, w, b)

(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)**2/6 + (b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)**2/6 + (b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)**2/6

In [26]:
compute_cost(Xn, yn, w, b)

(b + 852*w_1 + 2*w_2 + w_3 + 35*w_4 - 178)**2/6 + (b + 1416*w_1 + 3*w_2 + 2*w_3 + 40*w_4 - 232)**2/6 + (b + 2104*w_1 + 5*w_2 + w_3 + 45*w_4 - 460)**2/6

In [27]:
# test with optimal parameters
compute_cost(Xn, yn, wn_best, bn_best)

1.55789044289666e-12

In [28]:
# works for simple linear regression
compute_cost(X[:,0], y, m1, b)

(b + m_1*x_{11} - y_1)**2/6 + (b + m_1*x_{21} - y_2)**2/6 + (b + m_1*x_{31} - y_3)**2/6

In [29]:
compute_cost(Xs, ys, m1, b)

(b + m_1 - 300)**2/4 + (b + 2*m_1 - 500)**2/4

In [30]:
# test with optimal parameters
compute_cost(Xs, ys, ms_best, bs_best)

0

### Speed Comparison of `compute_cost_loop` and `compute_cost`

#### `compute_cost_loop`

In [31]:
%%timeit -r7 -n1000
compute_cost_loop(X, y, w, b)

2.48 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [32]:
%%timeit -r7 -n1000
compute_cost_loop(Xn, yn, w, b)

1.5 ms ± 251 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [33]:
%%timeit -r7 -n1000
compute_cost_loop(Xn, yn, wn_best, bn_best)

1.93 ms ± 338 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [34]:
%%timeit -r7 -n1000
compute_cost_loop(Xs, ys, ms_best, bs_best)

672 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### `compute_cost`

In [35]:
%%timeit -r7 -n1000
compute_cost(X, y, w, b)

375 µs ± 13.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [36]:
%%timeit -r7 -n1000
compute_cost(Xn, yn, w, b)

587 µs ± 132 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [37]:
%%timeit -r7 -n1000
compute_cost(Xn, yn, wn_best, bn_best)

721 µs ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [38]:
%%timeit -r7 -n1000
compute_cost(Xs, ys, ms_best, bs_best)

205 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## The Gradient

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})
\end{align}
$$

In [39]:
def compute_gradient_loop(X, y, w, b):
    m, n = X.shape

    dj_dw = sym.zeros(n, 1)
    dj_db = 0.0

    for i in range(m):
        err = f_wb(X[i,:], w, b)[0] - y[i]
        for j in range(n):
            dj_dw[j] = dj_dw[j] + (err * X[i, j])
        dj_db = dj_db + err
    dj_dw = dj_dw / m
    dj_db = dj_db / m

    return dj_db, dj_dw

In [40]:
# works for mulitiple linear regression
dj_db, dj_dw = compute_gradient_loop(X, y, w, b)
display(dj_db.factor())
display(dj_dw)

(3*b + w_1*x_{11} + w_1*x_{21} + w_1*x_{31} + w_2*x_{12} + w_2*x_{22} + w_2*x_{32} + w_3*x_{13} + w_3*x_{23} + w_3*x_{33} + w_4*x_{14} + w_4*x_{24} + w_4*x_{34} - y_1 - y_2 - y_3)/3

Matrix([
[x_{11}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{21}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{31}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{12}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{22}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{32}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{13}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{23}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{33}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{14}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{24}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{34}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3]])

In [41]:
dj_db, dj_dw = compute_gradient_loop(Xn, yn, w, b)
display(dj_db.factor())
display(dj_dw)

(3*b + 4372*w_1 + 10*w_2 + 4*w_3 + 120*w_4 - 870)/3

Matrix([
[4372*b/3 + 7157776*w_1/3 + 16472*w_2/3 + 5788*w_3/3 + 60380*w_4 - 1448008/3],
[            10*b/3 + 16472*w_1/3 + 38*w_2/3 + 13*w_3/3 + 415*w_4/3 - 3352/3],
[                 4*b/3 + 5788*w_1/3 + 13*w_2/3 + 2*w_3 + 160*w_4/3 - 1102/3],
[              40*b + 60380*w_1 + 415*w_2/3 + 160*w_3/3 + 4850*w_4/3 - 12070]])

In [42]:
# test with optimal parameters
dj_db, dj_dw = compute_gradient_loop(Xn, yn, wn_best, bn_best)
display(dj_db)
display(dj_dw)

-1.67392515019552e-6

Matrix([
[-0.00272623577196403],
[-6.27197262777675e-6],
[-2.21745578225333e-6],
[-6.92403390682254e-5]])

In [43]:
# works for simple linear regression
dj_db, dj_dw = compute_gradient_loop(X[:,0], y, m1, b)
display(dj_db.factor())
display(dj_dw)

(3*b + m_1*x_{11} + m_1*x_{21} + m_1*x_{31} - y_1 - y_2 - y_3)/3

Matrix([[x_{11}*(b + m_1*x_{11} - y_1)/3 + x_{21}*(b + m_1*x_{21} - y_2)/3 + x_{31}*(b + m_1*x_{31} - y_3)/3]])

In [44]:
dj_db, dj_dw = compute_gradient_loop(Xs, ys, m1, b)
display(dj_db.factor())
display(dj_dw)

(2*b + 3*m_1 - 800)/2

Matrix([[3*b/2 + 5*m_1/2 - 650]])

In [45]:
# test with optimal parameters
dj_db, dj_dw = compute_gradient_loop(Xs, ys, ms_best, bs_best)
display(dj_db)
display(dj_dw)

0

Matrix([[0]])

This can also be implemented without the loop.

In [46]:
def compute_gradient(X, y, w, b):
    m, n = X.shape

    y_pred = f_wb(X, w, b)
    err = y_pred - y

    dj_dw = (X.T @ err) / m
    dj_db = sum(err) / m

    return dj_db, dj_dw

In [47]:
# works for mulitiple linear regression
dj_db, dj_dw = compute_gradient(X, y, w, b)
display(dj_db.factor())
display(dj_dw)

(3*b + w_1*x_{11} + w_1*x_{21} + w_1*x_{31} + w_2*x_{12} + w_2*x_{22} + w_2*x_{32} + w_3*x_{13} + w_3*x_{23} + w_3*x_{33} + w_4*x_{14} + w_4*x_{24} + w_4*x_{34} - y_1 - y_2 - y_3)/3

Matrix([
[x_{11}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{21}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{31}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{12}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{22}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{32}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{13}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{23}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{33}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{14}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{24}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{34}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3]])

In [48]:
dj_db, dj_dw = compute_gradient(Xn, yn, w, b)
display(dj_db.factor())
display(dj_dw)

(3*b + 4372*w_1 + 10*w_2 + 4*w_3 + 120*w_4 - 870)/3

Matrix([
[4372*b/3 + 7157776*w_1/3 + 16472*w_2/3 + 5788*w_3/3 + 60380*w_4 - 1448008/3],
[            10*b/3 + 16472*w_1/3 + 38*w_2/3 + 13*w_3/3 + 415*w_4/3 - 3352/3],
[                 4*b/3 + 5788*w_1/3 + 13*w_2/3 + 2*w_3 + 160*w_4/3 - 1102/3],
[              40*b + 60380*w_1 + 415*w_2/3 + 160*w_3/3 + 4850*w_4/3 - 12070]])

In [49]:
# test with optimal parameters
dj_db, dj_dw = compute_gradient(Xn, yn, wn_best, bn_best)
display(dj_db)
display(dj_dw)

-1.67392515019552e-6

Matrix([
[-0.00272623577196403],
[-6.27197262777675e-6],
[-2.21745578225333e-6],
[-6.92403390682254e-5]])

In [50]:
# works for simple linear regression
dj_db, dj_dw = compute_gradient(X[:,0], y, m1, b)
display(dj_db.factor())
display(dj_dw)

(3*b + m_1*x_{11} + m_1*x_{21} + m_1*x_{31} - y_1 - y_2 - y_3)/3

Matrix([[x_{11}*(b + m_1*x_{11} - y_1)/3 + x_{21}*(b + m_1*x_{21} - y_2)/3 + x_{31}*(b + m_1*x_{31} - y_3)/3]])

In [51]:
dj_db, dj_dw = compute_gradient(Xs, ys, m1, b)
display(dj_db.factor())
display(dj_dw)

(2*b + 3*m_1 - 800)/2

Matrix([[3*b/2 + 5*m_1/2 - 650]])

In [52]:
# test with optimal parameters
dj_db, dj_dw = compute_gradient(Xs, ys, ms_best, bs_best)
display(dj_db)
display(dj_dw)

0

Matrix([[0]])

### Speed Comparison of `compute_gradient_loop` and `compute_gradient`

#### `compute_gradient_loop`

In [53]:
%%timeit -r7 -n1000
compute_gradient_loop(X, y, w, b)

1.16 ms ± 249 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [54]:
%%timeit -r7 -n1000
compute_gradient_loop(Xn, yn, w, b)

1.19 ms ± 228 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [55]:
%%timeit -r7 -n1000
compute_gradient_loop(Xn, yn, wn_best, bn_best)

1.72 ms ± 218 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [56]:
%%timeit -r7 -n1000
compute_gradient_loop(Xs, ys, ms_best, bs_best)

465 µs ± 103 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### `compute_gradient`

In [57]:
%%timeit -r7 -n1000
compute_gradient(X, y, w, b)

524 µs ± 13.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [58]:
%%timeit -r7 -n1000
compute_gradient(Xn, yn, w, b)

812 µs ± 203 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [59]:
%%timeit -r7 -n1000
compute_gradient(Xn, yn, wn_best, bn_best)

1.06 ms ± 191 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [60]:
%%timeit -r7 -n1000
compute_gradient(Xs, ys, ms_best, bs_best)

214 µs ± 5.65 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Using `sympy` to compute the gradient

This only works when the parameters, $\mathbf{w}$ and $b$ are variables.

In [61]:
def compute_gradient_sympy(X, y, w, b):
    cost = compute_cost(X, y, w, b)
    w = sym.Matrix([w]) # so it works with simple regression

    dj_dw = w.applyfunc(lambda x: sym.diff(cost, x))
    dj_db = sym.diff(cost, b)

    return dj_db, dj_dw

In [62]:
# works for mulitiple linear regression
dj_db, dj_dw = compute_gradient_sympy(X, y, w, b)
display(dj_db.factor())
display(dj_dw)

(3*b + w_1*x_{11} + w_1*x_{21} + w_1*x_{31} + w_2*x_{12} + w_2*x_{22} + w_2*x_{32} + w_3*x_{13} + w_3*x_{23} + w_3*x_{33} + w_4*x_{14} + w_4*x_{24} + w_4*x_{34} - y_1 - y_2 - y_3)/3

Matrix([
[x_{11}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{21}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{31}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{12}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{22}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{32}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{13}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{23}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{33}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3],
[x_{14}*(b + w_1*x_{11} + w_2*x_{12} + w_3*x_{13} + w_4*x_{14} - y_1)/3 + x_{24}*(b + w_1*x_{21} + w_2*x_{22} + w_3*x_{23} + w_4*x_{24} - y_2)/3 + x_{34}*(b + w_1*x_{31} + w_2*x_{32} + w_3*x_{33} + w_4*x_{34} - y_3)/3]])

In [63]:
dj_db, dj_dw = compute_gradient_sympy(Xn, yn, w, b)
display(dj_db.factor())
display(dj_dw)

(3*b + 4372*w_1 + 10*w_2 + 4*w_3 + 120*w_4 - 870)/3

Matrix([
[4372*b/3 + 7157776*w_1/3 + 16472*w_2/3 + 5788*w_3/3 + 60380*w_4 - 1448008/3],
[            10*b/3 + 16472*w_1/3 + 38*w_2/3 + 13*w_3/3 + 415*w_4/3 - 3352/3],
[                 4*b/3 + 5788*w_1/3 + 13*w_2/3 + 2*w_3 + 160*w_4/3 - 1102/3],
[              40*b + 60380*w_1 + 415*w_2/3 + 160*w_3/3 + 4850*w_4/3 - 12070]])

In [64]:
# works for simple linear regression
dj_db, dj_dw = compute_gradient_sympy(X[:,0], y, m1, b)
display(dj_db.factor())
display(dj_dw)

(3*b + m_1*x_{11} + m_1*x_{21} + m_1*x_{31} - y_1 - y_2 - y_3)/3

Matrix([[x_{11}*(b + m_1*x_{11} - y_1)/3 + x_{21}*(b + m_1*x_{21} - y_2)/3 + x_{31}*(b + m_1*x_{31} - y_3)/3]])

In [65]:
dj_db, dj_dw = compute_gradient_sympy(Xs, ys, m1, b)
display(dj_db.factor())
display(dj_dw)

(2*b + 3*m_1 - 800)/2

Matrix([[3*b/2 + 5*m_1/2 - 650]])

#### Speed Comparison of `compute_gradient` and `compute_gradient_sympy`

`compute_gradient_sympy` is much slower as expected.

##### `compute_gradient`

In [66]:
%%timeit -r7 -n1000
compute_gradient(X, y, w, b)

669 µs ± 146 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [67]:
%%timeit -r7 -n1000
compute_gradient(Xn, yn, w, b)

566 µs ± 12.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [68]:
%%timeit -r7 -n1000
compute_gradient(X[:,0], y, m1, b)

352 µs ± 8.78 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [69]:
%%timeit -r7 -n1000
compute_gradient(Xs, ys, m1, b)

323 µs ± 9.51 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


##### `compute_gradient_sympy`

In [70]:
%%timeit -r7 -n1000
compute_gradient_sympy(X, y, w, b)

1.93 ms ± 339 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [71]:
%%timeit -r7 -n1000
compute_gradient_sympy(Xn, yn, w, b)

1.66 ms ± 392 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [72]:
%%timeit -r7 -n1000
compute_gradient_sympy(X[:,0], y, m1, b)

939 µs ± 198 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [73]:
%%timeit -r7 -n1000
compute_gradient_sympy(Xs, ys, m1, b)

646 µs ± 18.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Gradient Descent

In [74]:
def gradient_descent(X, y, w, b, f_cost, f_gradient, alpha, num_iters):

    w = sym.Matrix([w]) # so it works with simple regression

    J_history = []

    for i in range(num_iters):
        dj_db, dj_dw = f_gradient(X, y, w, b)

        w = w - alpha * dj_dw
        b = b - alpha * dj_db

        J_history.append(f_cost(X, y, w, b))

        # print cost
        if i % ceil(num_iters / 10) == 0:
            print(f'Iteration {i:4d}: Cost {J_history[-1]:8.2f}')

    return w, b, J_history

This is no point in using the symbolic dataset here, as it results in very complicated algebraic expressions for the cost.

In [75]:
# test multiple linear regression with `compute_cost_loop` and `compute_gradient_loop`
initial_w = sym.zeros(w.shape[0], 1)
initial_b = 0.0
iterations = 1000
alpha = 5.0e-7

w_final, b_final, J_history = gradient_descent(Xn, yn,
                                               initial_w, initial_b,
                                               compute_cost_loop, compute_gradient_loop,
                                               alpha, iterations)

print('w_final:')
display(w_final)
print('b_final:')
display(b_final)
print('Cost:')
J_history[-1]

Iteration    0: Cost  2529.46
Iteration  100: Cost   695.99
Iteration  200: Cost   694.92
Iteration  300: Cost   693.86
Iteration  400: Cost   692.81
Iteration  500: Cost   691.77
Iteration  600: Cost   690.73
Iteration  700: Cost   689.71
Iteration  800: Cost   688.70
Iteration  900: Cost   687.69
w_final:


Matrix([
[  0.203965687318831],
[0.00374919220982854],
[-0.0112487038789788],
[-0.0658613999237372]])

b_final:


-0.00223540753093253

Cost:


686.703411666521

In [76]:
# test multiple linear regression with `compute_cost` and `compute_gradient`
initial_w = sym.zeros(w.shape[0], 1)
initial_b = 0.0
iterations = 1000
alpha = 5.0e-7

w_final, b_final, J_history = gradient_descent(Xn, yn,
                                               initial_w, initial_b,
                                               compute_cost, compute_gradient,
                                               alpha, iterations)

print('w_final:')
display(w_final)
print('b_final:')
display(b_final)
print('Cost:')
J_history[-1]

Iteration    0: Cost  2529.46
Iteration  100: Cost   695.99
Iteration  200: Cost   694.92
Iteration  300: Cost   693.86
Iteration  400: Cost   692.81
Iteration  500: Cost   691.77
Iteration  600: Cost   690.73
Iteration  700: Cost   689.71
Iteration  800: Cost   688.70
Iteration  900: Cost   687.69
w_final:


Matrix([
[  0.203965687318831],
[0.00374919220982854],
[-0.0112487038789788],
[-0.0658613999237372]])

b_final:


-0.00223540753093253

Cost:


686.703411666521

In [77]:
# test simple linear regression
initial_w = 0
initial_b = 0
iterations = 10_000
alpha = 1.0e-2
w_final, b_final, J_history = gradient_descent(Xs, ys,
                                               initial_b, initial_w,
                                               compute_cost, compute_gradient,
                                               alpha, iterations)

print('w_final:')
display(w_final)
print('b_final:')
display(b_final)
print('Cost:')
J_history[-1]

Iteration    0: Cost 79274.81
Iteration 1000: Cost     3.41
Iteration 2000: Cost     0.79
Iteration 3000: Cost     0.18
Iteration 4000: Cost     0.04
Iteration 5000: Cost     0.01
Iteration 6000: Cost     0.00
Iteration 7000: Cost     0.00
Iteration 8000: Cost     0.00
Iteration 9000: Cost     0.00
w_final:


Matrix([[199.992850751318]])

b_final:


100.011567727362

Cost:


6.74501466258040e-6

### Speed Comparision

We can be sure that `gradient_descent` runs faster when we use cost and gradient functions that use vectorization, but let's test it just for fun. To do so, we use a version of `gradient_descent` that suppresses printing.

In [78]:
def gradient_descent_no_print(X, y, w, b, f_cost, f_gradient, alpha, num_iters):

    w = sym.Matrix([w]) # so it works with simple regression

    for i in range(5):
        dj_db, dj_dw = f_gradient(X, y, w, b)

        w = w - alpha * dj_dw
        b = b - alpha * dj_db

        cost = f_cost(X, y, w, b)

    return w, b, cost

In [79]:
initial_w = sym.zeros(w.shape[0], 1)
initial_b = 0.0
iterations = 1000
alpha = 5.0e-7

In [80]:
%%timeit -r3 -n20
gradient_descent_no_print(Xn, yn,
                          initial_w, initial_b,
                          compute_cost_loop,
                          compute_gradient_loop,
                          alpha, iterations)

29.1 ms ± 217 µs per loop (mean ± std. dev. of 3 runs, 20 loops each)


In [81]:
%%timeit -r3 -n20
gradient_descent_no_print(Xn, yn,
                          initial_w, initial_b,
                          compute_cost,
                          compute_gradient,
                          alpha, iterations)

12.3 ms ± 3.55 ms per loop (mean ± std. dev. of 3 runs, 20 loops each)
